In [48]:
import kfp.components as comp
import kfp.v2 as kfp
import kfp.v2.dsl as dsl
from kfp.v2 import compiler
from kfp.v2.dsl import (
    component,
    Input,
    Output,
    HTML
)

from typing import NamedTuple

@component(
    packages_to_install=[], 
    base_image='python:3.9',
)
def html_get_comp( 
    http_url: str,
    report: Output[HTML]
)  -> None:
    
    import urllib.request

    fp = urllib.request.urlopen(http_url)
    mybytes = fp.read()

    mystr = mybytes.decode("utf8")
    fp.close()

    #print(mystr)
    with open(report.path, 'w') as f:
        f.write(mystr)
    
    return 

In [49]:
from unittest.mock import Mock, MagicMock, patch
from unittest import mock

report = Mock(spec=HTML, path = "./report.html")

html_get_comp.python_func(
        http_url="http://www.python.org", 
        report=report)

In [56]:
from kfp.dsl.types import GCPProjectID
@dsl.pipeline(
  name='html-get-pipeline',
  description='pipeline that gets an HTML content and dumpts it into an artefact',
)
def html_pipeline(
    http_url: str="http://www.python.org"
):
    
    http_url_op = html_get_comp(http_url= http_url)
    

In [57]:

compiler.Compiler().compile(
        pipeline_func=html_pipeline,
        package_path="html_pipeline.json"
)